# Biopython coronavirus notebook tutorial

This basic tutorial shows you how to identify an "Unknown sequence" of DNA/RNA, which happens to derive from a cornavirus genome (spoiler alert!). This tutorial uses [Biopython](https://github.com/biopython/biopython) (calling some tools) to identify and start to characterize this sequence.

## Setup

Imports and print version information

In [ ]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass

In [ ]:
import os
import sys

from urllib.request import urlretrieve
#
import Bio
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable

print("Python version:", sys.version_info)
print("Biopython version:", Bio.__version__)

Python version: sys.version_info(major=3, minor=8, micro=2, releaselevel='final', serial=0)
Biopython version: 1.76


Input file

In [ ]:
input_file = "unknown-sequence.fa"

fasta_loc = ("https://raw.githubusercontent.com/chris-rands/"
             "biopython-coronavirus/master/unknown-sequence.fa")

if not os.path.exists(input_file):
    urlretrieve(fasta_loc, input_file)

## Basic genome properties

In [ ]:
for record in SeqIO.parse(input_file, "fasta"):
    print(record.id)

Unknown_sequence


There is just a single sequence with header "Unknown_sequence". We are not dealing with many chromosomes, scaffolds or contigs.

Extract the sequence

In [ ]:
record = SeqIO.read(input_file, "fasta")

In [ ]:
record.seq

Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', SingleLetterAlphabet())

In [ ]:
print("Sequence length (bp)", len(record))

Sequence length (bp) 29903


The sequence length is ~30Kb, if this sequence represents an individual organism then it is very small. Far too small for a typical eukaryote and in fact too short many microbes too (e.g. bacterial genomes are typically Mb). This indicates that the genome could be from a virus.

In [ ]:
print("GC content (%)", GC(record.seq))

GC content (%) 37.97277865097148


The GC content is 0.38, so the sequence is somewhat AT-rich, but within a 'normal' range.

## Compare to other genome sequences

Let's use BLAST to align the unknown sequence to other annoated sequences in the NCBI nt database, which contains sequences from many different species from accross the tree of life.

This may take ~10 minutes since we are doing an online search against many sequences (for larger queries, it would sensible to run BLAST locally instead; see `Bio.Blast.Applications`)

In [ ]:
%%time
result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)

CPU times: user 155 ms, sys: 69 ms, total: 224 ms
Wall time: 2min 5s


Let's process the results with one of Biopython's generic parser

In [ ]:
blast_qresult = SearchIO.read(result_handle, "blast-xml")

In [ ]:
print(blast_qresult)

Program: blastn (2.10.0+)
  Query: No (29903)
         definition line
 Target: nt
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gi|1798174254|ref|NC_045512.2|  Wuhan seafood market pn...
            1      1  gi|1805293633|gb|MT019531.1|  Severe acute respiratory ...
            2      1  gi|1805293611|gb|MT019529.1|  Severe acute respiratory ...
            3      1  gi|1802633808|gb|MN996528.1|  Severe acute respiratory ...
            4      1  gi|1808633715|gb|MT049951.1|  Severe acute respiratory ...
            5      1  gi|1805293644|gb|MT019532.1|  Severe acute respiratory ...
            6      1  gi|1800455117|gb|MN988668.1|  Severe acute respiratory ...
            7      1  gi|1807860439|gb|MT039890.1|  Severe acute respiratory ...
            8      1  gi|1805293655|gb|MT019533.1|  Severe acute res

Those descriptions are truncated, let's view them in full, for just the first 5 records

In [ ]:
[hit.description for hit in blast_qresult[:5]]

['Wuhan seafood market pneumonia virus isolate Wuhan-Hu-1, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate BetaCoV/Wuhan/IPBCAMS-WH-03/2019, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate BetaCoV/Wuhan/IPBCAMS-WH-01/2019, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate WIV04, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/Yunnan-01/human/2020/CHN, complete genome']

Well that looks fairly conclusive, without doing any quantitative analyses, it's already very likely we have a coronavirus genome here, specifically SARS2-CoV-2 that was the cause of the COVID19 pandemic!

Let's have a look at the first result in a bit more detail to check some of the alignment metrics

In [ ]:
first_hit = blast_qresult[0]

In [ ]:
first_hit.description

'Wuhan seafood market pneumonia virus isolate Wuhan-Hu-1, complete genome'

In [ ]:
first_hsp = first_hit[0]
print(first_hsp.evalue, first_hsp.bitscore)

0.0 53927.4


In [ ]:
print(first_hsp.aln)

DNAAlphabet() alignment with 2 rows and 29903 columns
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTC...AAA No
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTC...AAA gi|1798174254|ref|NC_045512.2|


The alignment appears of high quality and not merely a spurious hit

We could view/save the full sequence alignment, for illustration purposes, here is just the first 100 characters in FASTA format

In [ ]:
print(first_hsp.aln.format("fasta")[:100])

>No definition line
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCT
GTTCTCTAAACGAACTTTA


## Extract annotations on the matching genome sequence

Let's extract a bit more structured meta-data on the top matching sequence homologous sequence using NCBI Entrez via Biopython to extract a GenBank file

In [ ]:
NCBI_id = first_hit.id.split('|')[3]
NCBI_id

'NC_045512.2'

In [ ]:
Entrez.email = "A.N.Other@example.com"  # Always tell NCBI who you are

In [ ]:
handle = Entrez.efetch(db="nucleotide", id= NCBI_id, retmode="text", rettype="gb")

In [ ]:
genbank_record = SeqIO.read(handle, "genbank")

In [ ]:
genbank_record

SeqRecord(seq=Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', IUPACAmbiguousDNA()), id='NC_045512.2', name='NC_045512', description='Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome', dbxrefs=['BioProject:PRJNA485481'])

There's a lot of information in the genbank record if you know where to find it...

In [ ]:
print("Is it single or double stranded and a DNA or RNA virus?\n",
      genbank_record.annotations["molecule_type"])

Is it single or double stranded and a DNA or RNA virus?
 ss-RNA


In [ ]:
print("What is the full NCBI taxonomy of this virus?\n",
      genbank_record.annotations["taxonomy"])

What is the full NCBI taxonomy of this virus?
 ['Viruses', 'Riboviria', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus']


In [ ]:
print("What are the relevant references/labs who generated the data?\n")
for reference in genbank_record.annotations["references"]:
    print(reference)

What are the relevant references/labs who generated the data?

location: [0:29903]
authors: Wu,F., Zhao,S., Yu,B., Chen,Y.-M., Wang,W., Hu,Y., Song,Z.-G., Tao,Z.-W., Tian,J.-H., Pei,Y.-Y., Yuan,M.L., Zhang,Y.-L., Dai,F.-H., Liu,Y., Wang,Q.-M., Zheng,J.-J., Xu,L., Holmes,E.C. and Zhang,Y.-Z.
title: A novel coronavirus associated with a respiratory disease in Wuhan of Hubei province, China
journal: Unpublished
medline id: 
pubmed id: 
comment: 

location: [0:29903]
authors: 
consrtm: NCBI Genome Project
title: Direct Submission
journal: Submitted (17-JAN-2020) National Center for Biotechnology Information, NIH, Bethesda, MD 20894, USA
medline id: 
pubmed id: 
comment: 

location: [0:29903]
authors: Wu,F., Zhao,S., Yu,B., Chen,Y.-M., Wang,W., Hu,Y., Song,Z.-G., Tao,Z.-W., Tian,J.-H., Pei,Y.-Y., Yuan,M.L., Zhang,Y.-L., Dai,F.-H., Liu,Y., Wang,Q.-M., Zheng,J.-J., Xu,L., Holmes,E.C. and Zhang,Y.-Z.
title: Direct Submission
journal: Submitted (05-JAN-2020) Shanghai Public Health Clinical Cent

Now we can read up more about the virus and source data through following these references and appropriate google searches.

Note that from this id, we could also find the [record here](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2/) on the NCBI website.

## Protein level analyses

We might be interested in the gene/protein sequences, not just the entire genome. It is possible to retrieve the protein coding sequences (CDSs) from the Genbank record

In [ ]:
len(genbank_record.features)

47

In [ ]:
{feature.type for feature in genbank_record.features}

{"3'UTR", "5'UTR", 'CDS', 'gene', 'mat_peptide', 'source', 'stem_loop'}

In [ ]:
CDSs = [feature for feature in genbank_record.features if feature.type == "CDS"]
len(CDSs)

12

Let's look at the first protein and extract the underlying sequence

In [ ]:
CDSs[0].qualifiers["gene"]

['orf1ab']

In [ ]:
protein_seq = Seq(CDSs[0].qualifiers["translation"][0])

In [ ]:
protein_seq

Seq('MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLV...VNN')

In [ ]:
print("Does the sequence begin with a start codon?\n",
      protein_seq.startswith("M"))

Does the sequence begin with a start codon?
 True


We can check roughly how this protein sequence relates to the underlying nucleotide sequence by looking at the DNA codon table.

In [ ]:
print(CodonTable.unambiguous_dna_by_id[1])

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

However, we can't perform an exact "reverse translation" of course, since several amino acids are produced by the same codon. Note that if instead we started with the nucleotide sequence, then we could use Biopython's `.transcribe()` and `.translate()` functions to convert sequences from DNA to RNA and DNA to protein respectively.

In [ ]:
print("Protein sequence length in amino acids", len(protein_seq))

Protein sequence length in amino acids 7096


It is a long protein for a virus. Let's check the annotation

In [ ]:
CDSs[0].qualifiers["product"]

['orf1ab polyprotein']

So it looks like this is a polyprotein, which explains why it is a relatively long protein. Polyproteins are a typical feature of some viral genomes where smaller proteins are joined together, providing a particular organization of the viral proteome.

## What's next?

Logical next steps at the genome level might include building a multiple sequence alignment from many cornavirus genomes (checkout the Biopython wrapers/parsers for `Clustal` and `Mafft` and `Bio.Align`/`Bio.parirwise2` plus `Bio.AlignIO`), building a phylogeny with an external tool like [iq-tree](http://www.iqtree.org/) and then viewing the tree with `Bio.Phylo`, the [ete3 toolkit](http://etetoolkit.org/), or [Jalview](https://www.jalview.org/).

Other protein level analyses could involve including building protein trees, annotating the proteins and vizulisation (e.g. `Bio.Graphics`), doing evolutionary rate analyses (e.g. `Bio.Phylo.PAML `), looking at protein structure, clustering and much more.

These kind of analyses can provide useful biological and epidemiological information, very important for this coronavirus in an outbreak situation. For example, allowing tracking of how the outbreak spreads and indicating appropriate infection control measures, although caution in the inturpretation of results is always required. See [Nextstrain](https://nextstrain.org/ncov) for an excellent resource of this kind.

Note, there is tons of other functionality in Biopython, this is just a very small fraction of the modules, see [Peter Cock's Biopython workshop](https://github.com/peterjc/biopython_workshop) and the extensive [official tutorial documentation](http://biopython.org/DIST/docs/tutorial/Tutorial.html).